In [10]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

We now get the data

In [11]:
wikisource = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(wikisource.text, 'lxml')

We now assess the data

In [17]:
#extract the data from the wikitable data 
data = []
columns = []
table = soup.find(class_= 'wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    #set first row as columns
    if (index == 0):
        columns = section
    else:
        data.append(section)
        
toronto_df = pd.DataFrame(data=data, columns=columns)
toronto_df.head()
        


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


We will now clean the data

In [18]:
toronto_df.rename(columns={"Postcode":"PostalCode", "Neighbourhood":"Neighborhood"}, inplace=True)
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [31]:
df1 = toronto_df
toronto_df1 = df1.loc[df1['Neighborhood']=='Not assigned']

for i in toronto_df1.index:
    toronto_df1.at[i, 'Neighborhood'] = toronto_df1.at[i, 'Borough']
    
df1.loc[toronto_df1.index] = toronto_df1

In [32]:
toronto_df = df1[df1.Borough != 'Not assigned']

In [35]:
toronto_df = toronto_df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: ','.join(x.astype(str))).reset_index()
toronto_df = toronto_df.drop_duplicates()
toronto_df.head(11)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"
